# Linear regression with sklearn API

Setup:

1. Dataset: California housing
2. Linear regression API: `LinearRegression`
3. Training: `fit`(normal equation) and `cross_validate`(normal equation with cross validation).
4. Evaluation: `score`($R^2$ Score) and `cross_val_score` with different scoring parameters.

We will study the model diagnosis with `LearningCurve` and learn how to examine the learned model or weight vector.

In [ ]:
#Libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict

from sklearn.metrics import mean_squared_error

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


In [ ]:
np.random.seed(306)
plt.style.use('seaborn')

We will use `ShuffleSplit` cross validation with:
* 10 folds (`n_splits`) and
* Set aside 20% examples as test examples(`test_size`) in each fold.

In [ ]:
shuffle_split_cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)

## **Step 1**: Load the dataset

In [ ]:
features, labels = fetch_california_housing(as_frame=True, return_X_y=True)

print("shape of feature matrix: ", features.shape)
print("shape of label matrix: ", labels.shape)

sanity check

In [ ]:
assert(features.shape[0] == labels.shape[0])

## **Step 2**: Data exploration

[Done in seperate notebook](California_housing_dataset_exploration.ipynb)

## **Step 3**: Preprocessing and model building

### 3.1 Train test split

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, random_state=42)

print("# training samples: ", train_features.shape[0])
print("# training samples: ", test_features.shape[0])

Sanity checks

In [ ]:
assert (train_features.shape[0] == train_labels.shape[0])
assert (test_features.shape[0] == test_labels.shape[0])

### 3.2 Pipeline: Preprocessing + Model

`Pipeline` object we are going to use have two components:
1. `StandardScaler`
2. `LinearRegression`

In [ ]:
lin_reg_pipeline = Pipeline([("feature_scaling", StandardScaler( )),
                            ("lin_reg", LinearRegression())])

lin_reg_pipeline.fit(train_features, train_labels)

Now that we have trained the model, let's check the learned/estimated weight vectors

In [ ]:
print("Intercept (w_0):", lin_reg_pipeline[-1].intercept_) # intercept through intercept_
print("Weight vector (w_1,....., w_m):", lin_reg_pipeline[-1].coef_) # rest of the weights through weight_

## **Step 4**: Model Evaluation

### `score`

* $R^2$ score 

In [ ]:
#Evaluate model performance in the test set.add
test_score = lin_reg_pipeline.score(test_features, test_labels)
print("Model performance on test set: ", test_score)

train_score = lin_reg_pipeline.score(train_features, train_labels)
print("Model performance on train set: ", train_score)

`r2` score is not high enough $=>$ underfitting

### Cross validated score(`cross_val_score`)

* Calculates `r2` on different folds through cross validation

In [ ]:
lin_reg_score = cross_val_score(lin_reg_pipeline,
                                train_features,
                                train_labels,
                                scoring = 'neg_mean_squared_error',
                                cv = shuffle_split_cv)

#This will print 10 different scores, one for each score
print(lin_reg_score)

# We can take the mean and standard deviation of the score and report it.
print(f"\nScore of linear regression model on the test set: \n"
        f"{lin_reg_score.mean():.3f} +/- {lin_reg_score.std():.3f}")

Other available 'scoring` parameters
* `explained_variance`
* `max_error`
* `neg_mean_absolute_error`
* `neg_root_mean_squared_error`
* `neg_mean_squared_log_error`
* `neg_median_absolute_error`
* `neg_mean_absolute_percentaage_error`
* `r2`

### Cross Validation

* `cross_validate` gives access to models trained in each fold along with some other statistics

In [ ]:
lin_reg_cv_results = cross_validate(lin_reg_pipeline,
                                    train_features,
                                    train_labels,
                                    cv = shuffle_split_cv,
                                    scoring="neg_mean_squared_error",
                                    return_train_score=True,
                                    return_estimator=True)

In [ ]:
lin_reg_cv_results

multiply these scores by -1 to convert them into errors

In [ ]:
train_error = -1 * lin_reg_cv_results['train_score']
test_error = -1 * lin_reg_cv_results['test_score']

print(f"Mean squareed error of linear regression model on the train set:\n"
        f"{train_error.mean():.3f} +/- {train_error.std():.3f}")

print(f"Mean squareed error of linear regression model on the test set:\n"
        f"{test_error.mean():.3f} +/- {test_error.std():.3f}")

* The training and test errors are high, which is an indication of underfitting, which we will confirm by plotting the learnig curve.
* Test error has high variability across different folds compared to the training error.

### Effect of training set size on error 

In [ ]:
# Function for visualisation

def plot_learning_curve(train_sizes, train_scores, test_scores):

    train_scores_mean = np.mean(-train_scores, axis=1)
    train_scores_std = np.std(-train_scores, axis=1)
    test_scores_mean = np.mean(-test_scores, axis=1)
    test_scores_std = np.std(-test_scores, axis=1)

    plt.fill_between(
            train_sizes,
            train_scores_mean - train_scores_std,
            train_scores_mean + train_scores_std,
            alpha = 0.1,
            color = "r",)

    plt.fill_between(
            train_sizes,
            test_scores_mean - test_scores_std,
            test_scores_mean + test_scores_std,
            alpha = 0.1,
            color = "g",)

    plt.plot(train_sizes, train_scores_mean, "o-", color="r", label="Training score")
    plt.plot(train_sizes, test_scores_mean, "o-", color="g", label="Cross-validation score")
    plt.xlabel("Training examples")
    plt.ylabel("MSE")
    plt.legend(loc="best")

Based on the scores calculated by `learning_curve` API, we plot the error and its standard deviation for different number of samples.

In [ ]:
(train_sizes, train_scores, test_scores, fit_times, score_times) = learning_curve(lin_reg_pipeline,
                                                                                    train_features,
                                                                                    train_labels,
                                                                                    cv=shuffle_split_cv,
                                                                                    scoring='neg_mean_squared_error',
                                                                                    n_jobs=-1,
                                                                                    return_times=True,
                                                                                    train_sizes=np.linspace(0.2, 1.0, 10))

plot_learning_curve(train_sizes, train_scores, test_scores)

observations:

* Both curves have reached a plateau" They are close and fairly high.
* Few instances in the training set mean the model can fit them perfectly. But as more instances are added to the training set, it becomes impossible for the model to fit the training data perfectly.
* When the model is trained on very few instances, it is not able to generalize properly, hence the validation error is high.

These learning curves are typical of underfitting model

### Model Examination

Let's examine the weight vectors and how much variability exists between them across different cross-validated models

In [ ]:
feature_names = train_features.columns
feature_names

In [ ]:
coefs = [est[-1].coef_ for est in lin_reg_cv_results["estimator"]]
weights_df = pd.DataFrame(coefs, columns=feature_names)

color = {"whiskers": "black", "medians": "black", "caps": "black"}
weights_df.plot.box(color=color, vert=False)
_ = plt.title("Linear regression coefficients")

There is not much variability in weights learned by different models. It can also be seen from the std deviation

In [ ]:
weights_df.describe()

### Selecting best model

In [ ]:
best_model_index = np.argmin(test_error)
selected_model = lin_reg_cv_results['estimator'][best_model_index]

In [ ]:
print("Intercept (w_0):", selected_model['lin_reg'].intercept_) # intercept through intercept_
print("Weight vector (w_1,....., w_m):", selected_model['lin_reg'].coef_) # rest of the weights through weight_

### Model performance


In [ ]:
cv_predictions = cross_val_predict(lin_reg_pipeline, train_features, train_labels)

mse_cv = mean_squared_error(train_labels, cv_predictions)

plt.scatter(train_labels, cv_predictions, color='blue')
plt.plot(train_labels, train_labels, 'r-')
plt.title(f"Mean squared error = {mse_cv:.2f}", size=24)
plt.xlabel("Actual Median House Value", size=15)
plt.ylabel("Predicted Median House Value", size=15)
plt.show()

At this stage, we should perform error analysis and check where the predictions are going wrong. We can revisit feature construction, preprocessing and model stages and make the necessary course correction to get better performance.

## STEP 5: Predictions

In [ ]:
test_predictions_cv = selected_model.predict(test_features)
test_predictions_cv[:5]

We can also obtain predictions using initial model that we built without cross validation

In [ ]:
test_predictions = lin_reg_pipeline.predict(test_features)
test_predictions[:5]

## STEP 6: Report model performance


In [ ]:
score_cv = selected_model.score(test_features, test_labels)
score = lin_reg_pipeline.score(test_features, test_labels)

print("R2 score for the best model obtained via cross validation: ", score_cv)
print("R2 score for the model without cross validation: ", score)

Cross validated models have slightly better results